# Import liberaies

In [2]:
from selenium import webdriver
import pandas as pd

# Scrape the full website pages

In [3]:
# locate and create instance from selenium webdriver
PATH = "chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(PATH)
# create list to save all book details
all_details = []
# scrape all the 50 pages
for i in range(1, 51):
    try:
        driver.get("http://books.toscrape.com/catalogue/category/books_1/page-{}.html".format(i))
        # select by the xpath
        incategory = driver.find_elements_by_class_name("product_pod")
        links = []
        # select all links in the page
        for j in range(len(incategory)):
            item = incategory[j]
            link = item.find_element_by_tag_name("h3").find_element_by_tag_name("a").get_property("href")
            links.append(link)
        # get all informations about the book in every link
        for link in links:
            driver.get(link)
            title = driver.find_element_by_xpath("//*[@id='content_inner']/article/div[1]/div[2]/h1")
            price = driver.find_element_by_xpath("//*[@id='content_inner']/article/div[1]/div[2]/p[1]")
            stock = driver.find_element_by_xpath("//*[@id='content_inner']/article/div[1]/div[2]/p[2]")
            stars = driver.find_element_by_xpath("//*[@id='content_inner']/article/div[1]/div[2]/p[3]").get_attribute("class")
            stars = stars.split()[1]
            category_a =  driver.find_element_by_xpath("//*[@id='default']/div/div/ul/li[3]/a")
            
            try:
                description = driver.find_element_by_xpath("//*[@id='content_inner']/article/p")
                description = description.text
            except:
                description = None
            # create a dictionary to save all book informations and append it to all_details list
            row = {
				"Title":title.text,
				"Category":category_a.text,
				"Stock": stock.text,
				"Stars": stars,
				"Price":price.text,
				"Description": description
			}
            all_details.append(row)
    except:
        driver.close()

# Create dataframe for all scrapped informations

In [4]:
all_books_df = pd.DataFrame(all_details)
all_books_df.head()

,Title,Category,Stock,Stars,Price,Description
0,A Light in the Attic,Poetry,In stock (22 available),Three,£51.77,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,Historical Fiction,In stock (20 available),One,£53.74,"""Erotic and absorbing...Written with starling ..."
2,Soumission,Fiction,In stock (20 available),One,£50.10,"Dans une France assez proche de la nôtre, un h..."
3,Sharp Objects,Mystery,In stock (20 available),Four,£47.82,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,History,In stock (20 available),Five,£54.23,From a renowned historian comes a groundbreaki...


# Save the dataframe to csv file

In [5]:
all_books_df.to_csv("all_books.csv", index=False)
# close the driver
driver.close()